In [1]:
from detectron2.engine.hooks import HookBase
from detectron2.evaluation import inference_context
from detectron2.utils.logger import log_every_n, log_every_n_seconds
from detectron2.data import DatasetMapper, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator
from detectron2.engine import DefaultTrainer
import detectron2.utils.comm as comm
import torch
import time
import datetime
import logging
import os
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
import matplotlib.pyplot as plt
import time
import yaml

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.utils.visualizer import ColorMode
from detectron2.structures import BoxMode

In [ ]:
class LossEvalHook(HookBase):
    def __init__(self, eval_period, model, data_loader):
        self._model = model
        self._period = eval_period
        self._data_loader = data_loader
    
    def _do_loss_eval(self):
        # Copying inference_on_dataset from evaluator.py
        total = len(self._data_loader)
        num_warmup = min(5, total - 1)
            
        start_time = time.perf_counter()
        total_compute_time = 0
        losses = []
#         with inference_context(self._model), torch.no_grad():
        for idx, inputs in enumerate(self._data_loader):            
            if idx == num_warmup:
                start_time = time.perf_counter()
                total_compute_time = 0
            start_compute_time = time.perf_counter()
            if torch.cuda.is_available():
                torch.cuda.synchronize()
            total_compute_time += time.perf_counter() - start_compute_time
            iters_after_start = idx + 1 - num_warmup * int(idx >= num_warmup)
            seconds_per_img = total_compute_time / iters_after_start
            if idx >= num_warmup * 2 or seconds_per_img > 5:
                total_seconds_per_img = (time.perf_counter() - start_time) / iters_after_start
                eta = datetime.timedelta(seconds=int(total_seconds_per_img * (total - idx - 1)))
                log_every_n(
                    logging.INFO,
                    "Loss on Validation  done {}/{}. {:.4f} s / img. ETA={}".format(
                        idx + 1, total, seconds_per_img, str(eta)),
                    n=5,
                )
                loss_batch = self._get_loss(inputs)
                losses.append(loss_batch)
        mean_loss = np.mean(losses)
        self.trainer.storage.put_scalar('validation_loss', mean_loss)
        comm.synchronize()

        return losses
            
    def _get_loss(self, data):
        # How loss is calculated on train_loop 
        metrics_dict = self._model(data)
        metrics_dict = {
            k: v.detach().cpu().item() if isinstance(v, torch.Tensor) else float(v)
            for k, v in metrics_dict.items()
        }
        total_losses_reduced = sum(loss for loss in metrics_dict.values())
        return total_losses_reduced
        
        
    def after_step(self):
        next_iter = self.trainer.iter + 1
        is_final = next_iter == self.trainer.max_iter
        if is_final or (self._period > 0 and next_iter % self._period == 0):
            self._do_loss_eval()
        self.trainer.storage.put_scalars(timetest=12)

In [ ]:
class CustomTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls,cfg,dataset_name, output_folder = None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR,"inference")
        return COCOEvaluator(dataset_name, cfg, True, output_dir = output_folder)
    
    def build_hooks(self):
        hooks = super().build_hooks()
        hooks.insert(-1,LossEvalHook(
            cfg.TEST.EVAL_PERIOD,
            self.model,
            build_detection_test_loader(
                self.cfg,
                self.cfg.DATASETS.TEST[0],
                DatasetMapper(self.cfg,True)
            )
        ))
        return hooks
    
        

In [ ]:
# if your dataset is in COCO format, this cell can be replaced by the following three lines:
# from detectron2.data.datasets import register_coco_instances
# register_coco_instances("my_dataset_train", {}, "json_annotation_train.json", "path/to/image/dir")
# register_coco_instances("my_dataset_val", {}, "json_annotation_val.json", "path/to/image/dir")


def create_output_dir(lr, batch_n):
    return f"lr={float(lr)}--batch_n={batch_n}_{int(time.time())}"


def export_cfg(cfg,name,output_dir):
    if "yaml" in name:
        with open(os.path.join(output_dir,name),"w") as file:
            yaml.dump(cfg,file)
    else:
        with open(os.path.join(output_dir,name+".yaml"),"w") as file:
            yaml.dump(cfg,file)
            
            
def get_internet_imgs():
    internet_names = os.listdir("/home/max/Desktop/dissertation/Mask_RCNN/barry_data/internet/")
    return ["/home/max/Desktop/dissertation/Mask_RCNN/barry_data/internet/"+file for file in internet_names]
            

# Extracting the LR and batch_size from the trained models filename
def get_batch_num(path):
    return float(path.split("=")[-1].split("_")[0])
def get_lr(path):
    return np.format_float_positional(float(path.split("=")[1].split("--")[0]))


def get_shoe_dicts(img_dir):
    json_file = os.path.join(img_dir, "via_region_data.json")
    print(json_file)
    with open(json_file) as f:
        imgs_anns = json.load(f)

    dataset_dicts = []
    for idx, v in enumerate(imgs_anns["_via_img_metadata"].values()):
        record = {}
        
        filename = os.path.join(img_dir, v["filename"])
        height, width = cv2.imread(filename).shape[:2]
        
        record["file_name"] = filename
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width
      
        annos = v["regions"]
        objs = []
        for anno in annos:
#             assert not anno["region_attributes"]
            anno = anno["shape_attributes"]
            px = anno["all_points_x"]
            py = anno["all_points_y"]
            poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
            poly = [p for x in poly for p in x]

            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": 0,
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

for d in ["train", "val"]:
    DatasetCatalog.register("shoe_" + d, lambda d=d: get_shoe_dicts("/home/max/Desktop/dissertation/Mask_RCNN/barry_data/" + d))
    MetadataCatalog.get("shoe_" + d).set(thing_classes=["shoe"])
    
shoe_metadata = MetadataCatalog.get("shoe_train")



In [ ]:
"""

Instantiate a model from a config file.

Can be passed a custom config in yaml format

"""


cfg = get_cfg()s
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("shoe_train",)
cfg.DATASETS.TEST = ("shoe_val",)
cfg.TEST.EVAL_PERIOD = 50
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0002  # pick a good LR

cfg.SOLVER.MAX_ITER = 600    # 300 iterations seems good enough for this toy dataset; you may need to train longer for a practical dataset
# cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 1
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (shoe)
cfg.PERIOD = 1




In [ ]:
# """
# change the path if neccessary

# """
cfg.OUTPUT_DIR = "./run_logs_test_custom4/"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CustomTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
plot_train_val_loss(cfg.OUTPUT_DIR)

In [4]:
for i in range(100,1000,200):
    print(i)

100
300
500
700
900


In [6]:
np.format_float_positional(7.2e-5)

'0.000072'

In [ ]:
class MaskedCreptron:
    """
    Creates an instance of a shoe segmentation model based on Facebooks Detectron2
    
    https://github.com/facebookresearch/detectron2/
    
    
    
    includes the loading and registering the dataset and metadata in the percerptron 
    
    
    
    
    parameters to pass:

    -    train/val/test - have them in 1 directory if 'test' then add it in the list to be registered
        
    -    OUTPUT_DIR - dir to store output
    
    -    CUSTOM_CONFIG - custom config cosisting of all of the model parameters
    
    
    
    TO DO FUNCTIONS:
   
    register the dataset and metadata 
    
   
    LAOD CFG (CUSTOM OR NOT)
   
    INSTANCIATE FROM CUSTOM (TO BE CHANGED TO BASED ON SIMPLE) TRAINER
    
    creating the losseval - improvwe it 
   
    
    
    
    """
    
    def __init__(self, custom_cfg):
      #metadata and data register    
        
        
        
        
    
    def 
        